In [4]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.utils import resample

%matplotlib inline
pd.options.display.max_columns = None
pd.options.display.max_rows = 100


In [5]:
df_raw_accepted = pd.read_csv('accepted_2007_to_2018Q4.csv.gz')
df_raw_rejected = pd.read_csv('rejected_2007_to_2018Q4.csv.gz')

C:\Users\Laptop2\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
'''
For applications prior to November 5, 2013 the risk score is the borrower's FICO score. 
For applications after November 5, 2013 the risk score is the borrower's Vantage score.
'''

def data_process(df_accepted,df_rejected):
    
    accepted = df_accepted.copy()
    rejected = df_rejected.copy()
    
    # Processing accepted data 
    accepted = accepted[['loan_amnt', 'purpose', 'dti',
       'zip_code', 'addr_state', 'emp_length', 'issue_d',
       'loan_status', 'last_fico_range_high','last_fico_range_low']]
    
    accepted['issue_d'] = pd.to_datetime(accepted['issue_d'])
    accepted['app_year'] = accepted['issue_d'].dt.year
    accepted['app_month'] = accepted['issue_d'].dt.month

    accepted['risk_score'] = (accepted['last_fico_range_high'] + accepted['last_fico_range_low'])/2
    accepted = accepted.drop(columns = ['issue_d','last_fico_range_high','last_fico_range_low'])

#     if reverse = False:
#         accepted = accepted.replace({'loan_status' : { 'Charged Off' : 'rejected', 'Late (16-30 days)' : 'rejected', 
#                                        'Late (31-120 days)' : 'rejected',
#                                        'Does not meet the credit policy. Status:Charged Off' : 'rejected',
#                                        'Fully Paid' : 'accepted', 'Current' : 'accepted',
#                                        'In Grace Period' : 'accepted',
#                                        'Does not meet the credit policy. Status:Fully Paid' : 'accepted'}})

    accepted['loan_status'] =1
            
    
    # Processing rejected data 
    rejected = rejected.iloc[:,1:9]
    rejected = rejected.rename(columns={"Amount Requested": "loan_amnt", "Application Date": "app_date", "Loan Title": "purpose",
                             "Risk_Score": "risk_score", "Debt-To-Income Ratio": "dti", "Zip Code": "zip_code",
                             "State": "addr_state", "Employment Length": "emp_length"})
    rejected['loan_status'] = 0
    
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*consolidation.*$)', 'debt_consolidation')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*loan.*$)', 'debt_consolidation')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*debt.*$)', 'debt_consolidation')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*card.*$)', 'credit_card')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*credit.*$)', 'credit_card')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*house.*$)', 'house')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*business.*$)', 'small_business')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*car.*$)', 'car')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*education.*$)', 'education')
    
    
    rejected['purpose'] = np.where(rejected['purpose'].isin(['house', 'credit_card', 'major_purchase', 'debt_consolidation',
       'other', 'moving', 'small_business', 'home_improvement',
       'vacation', 'medical', 'renewable_energy', 'car', 'wedding']), rejected['purpose'], 'other')

    rejected['app_date'] = pd.to_datetime(rejected['app_date'])
    rejected['app_year'] = rejected['app_date'].dt.year
    rejected['app_month'] = rejected['app_date'].dt.month

    rejected = rejected.drop(columns = 'app_date')
    
    rejected['dti'] = rejected['dti'].str.replace('%','')
    
    # Merge accepted and rejected data for later process
    df_processed = pd.concat([accepted, rejected], sort=True)
    
    #Drop zip Code
    df_processed=df_processed.drop(['zip_code'],axis=1)
    
    # Convert categorical to numerical-- 10 means more than 10 years 
    df_processed['emp_length'] = df_processed['emp_length'].str.extract('(\d+)')


    # Missing Values: risk_score, emp_length, dti
    df_processed['emp_length'] = df_processed['emp_length'].fillna(0)
    df_processed['emp_length'] = df_processed['emp_length'].astype(int)
    df_processed['risk_score'] = df_processed['risk_score'].fillna(0)
    df_processed['dti'] = df_processed['dti'].fillna(0)
    df_processed['dti'] = df_processed['dti'].astype(float)
    
    df_processed = df_processed.drop(['purpose'], axis = 1)
    df_processed = pd.get_dummies(df_processed)
    
    
    return df_processed


In [8]:
#Data Process the Raw Data
df_processed = data_process(df_raw_accepted,df_raw_rejected)


MemoryError: 

In [ ]:
# Handling imbalanced data by downsize the majority from the original dataset

# The predicted column is "loan_status" , split the data into training and test sets.
#test set will  be used for final model accuracy test
#train set will be used for testing models

x_m = df_processed.drop(['loan_status'], axis=1)
y_m = df_processed.loc[:,'loan_status']
X_train, X_test, y_train, y_test = train_test_split(x_m, y_m, test_size=0.2, random_state=42)

#downsampling train set
# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
reject=X[X.loan_status==0] #reject
accept=X[X.loan_status==1]  #accept

reject_downsampled = resample(reject,
                                replace = False, # sample without replacement
                                n_samples = len(accept), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([reject_downsampled, accept])
y_train_balanced = downsampled.loan_status
X_train_balanced = downsampled.drop('loan_status', axis=1)

y_train_d.to_csv('balanced_y_train.csv')
X_train_d.to_csv('balanced_x_train.csv')
X_test.to_csv('unbalanced_X_test')
y_test.to_csv('unbalanced_y_test')




In [ ]:
#downsample test set

# concatenate our test data back together
X_test = pd.concat([X_test, y_test], axis=1)

# separate minority and majority classes
reject_test=X_test[X_test.loan_status==0] #reject
accept_test=X_test[X_test.loan_status==1]  #accept

reject_downsampled_test = resample(reject_test,
                                replace = False, # sample without replacement
                                n_samples = len(accept_test), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled_test = pd.concat([reject_downsampled_test, accept_test])
y_test_balanced = downsampled.loan_status
X_test_balanced = downsampled.drop('loan_status', axis=1)
y_test_balanced.to_csv('balanced_y_test')
X_test_balanced.to_csv('balanced_X_test')